In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import requests
import bs4
from bs4 import BeautifulSoup


import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("./seoul_food.csv", encoding='cp949')

In [3]:
df.head()

,구,동,사업장명,업종,좌표정보(X),좌표정보(Y)
0,종로구,관철동,상해,분식,198867.702591,451824.281126
1,관악구,신림동,미쿠,일식,193687.348199,442284.164025
2,구로구,구로동,여름엔빙수겨울엔떡국,한식,189814.305919,443046.239859
3,강남구,대치동,카페디퍼,카페,204776.548264,444677.642985
4,동대문구,신설동,메이크데이(MakeDay),카페,202114.228397,452392.093759


### requests과 공공데이터를 이용해서 리뷰주소 크롤링

In [150]:
id_list = []
cnt = 0
for i in range(len(id_list),df.shape[0]):
    keyword = df['구'][i] +' '+ df['동'][i] +' '+ df['사업장명'][i]
    url = f'https://search.map.kakao.com/mapsearch/map.daum?callback=jQuery18105963491453995977_1649742498249&q={keyword}&msFlag=A&sort=0'
    header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
        'referer': 'https://map.kakao.com/'
    }
    response = requests.get(url, headers=header)
    try:
        confirmid = response.text.split('"confirmid":"')[1].split('","x":')[0]
        review_address = f'https://place.map.kakao.com/{confirmid}#comment'
        id_list.append(review_address)
        cnt += 1
    except:
        id_list.append('error')
        cnt += 1
    if cnt % 1000 == 0:
        print(f'{cnt}개 진행완료')

106000개 진행완료
107000개 진행완료
108000개 진행완료
109000개 진행완료
110000개 진행완료
111000개 진행완료
112000개 진행완료
113000개 진행완료
114000개 진행완료


In [157]:
Counter(id_list)['error']

22715

In [154]:
df.insert(6,'리뷰주소',id_list)

In [155]:
df

,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소
0,종로구,관철동,상해,분식,198867.702591,451824.281126,https://place.map.kakao.com/13333296#comment
1,관악구,신림동,미쿠,일식,193687.348199,442284.164025,https://place.map.kakao.com/658611073#comment
2,구로구,구로동,여름엔빙수겨울엔떡국,한식,189814.305919,443046.239859,https://place.map.kakao.com/1160052313#comment
3,강남구,대치동,카페디퍼,카페,204776.548264,444677.642985,https://place.map.kakao.com/637651089#comment
4,동대문구,신설동,메이크데이(MakeDay),카페,202114.228397,452392.093759,error
...,...,...,...,...,...,...,...
114050,성동구,행당동,두마리찜닭 두찜 한양대점,한식,203554.866076,450449.723494,error
114051,동작구,신대방동,우동짜장&야식포차,해산물,191735.419996,442882.907854,https://place.map.kakao.com/20604687#comment
114052,중구,신당동,금성회관 약수역직영점,한식,200914.227438,450426.403396,error
114053,광진구,화양동,블랙아웃(Black out),술집,205812.137123,449038.140512,https://place.map.kakao.com/581196091#comment


In [156]:
df.to_csv('./seoul_food_id.csv',encoding='cp949',index=False)